In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import getpass  # to get the password without showing the input
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

········


## Instructions

In this ~bonus lab~ business case study, you will be using the Sakila database of movie rentals. Choose one of the cases below, or both if you dare:

**Case 1:**

We will be trying to predict if a **customer will be renting a film** this month based on their previous activity and other details. We will first construct a table with:

    Customer ID
    City
    Most rented film category
    Total films rented
    Total money spent
    How many films rented last month (MAY/2005)
    If the customer rented a movie this month (JUNE/2005)

Once you extract that information, and any other that seems fit, predict which customers will be renting this month.

**Case 2:**

We will be trying to predict if a **film will be rented** this month based on their previous activity and other details. We will first construct a table with:

    Film ID
    Category
    Total number of copies
    *Bonus - How many "stars" actrs. in the film*
    How many times the film was rented last month (MAY/2005)
    If the film was rented this month (JUNE/2005)

Once you extract that information, and any other that seems fit, predict which films will be rented this month.

**Suggestions:**

Format your notebook as a report, in a way that someone who didn't know your project could read and understand. Write down your hypothesis, what data you think it's important to check the hypothesis.

Have a notebook with general EDA, but on the final notebook have only the focused EDA.

Explain any transformation/scaling/feature engineering you apply (no need to explain data cleaning steps).

Write some conclusions: how would this prediction impact the business? what are your constraints in this business case? is it a good predictive model? any problems you suspect of? any ideas on how to solve the problem and/or improve the model?

**Bonus - Challenge:** Choose another target variable and explore other possibilities of predictions. Make sure to explain your business application and hypothesis.

**Delivery:** March 11th - Thursday

Create a huge query and create a table predict_rentals in SQL with row number for the most rented category to get the columns requested, see below:

create table if not exists predict_rentals
select cust.customer_id, city.city, category.name as category_name, count(r.rental_id) as number_of_rentals,sum(p.amount) as Total_EUR,row_number() OVER (PARTITION BY cust.customer_id ORDER BY count(category_name) DESC) AS rank_category_per_customer from rental r join payment p using(rental_id) join customer cust on p.customer_id=cust.customer_id join address a on a.address_id=cust.address_id join inventory i on i.inventory_id=r.inventory_id join film f on f.film_id=i.film_id join film_category cat on f.film_id=cat.film_id join category category on cat.category_id=category.category_id join city city on city.city_id=a.city_id
group by cust.customer_id,city.city,category.name order by cust.customer_id desc;

Create a 2nd table on basis of predict_rentals to filter to most rented category and name predict_rentals_step_two:

create table if not exists predict_rentals_step_two
select customer_id,city,category_name,number_of_rentals,Total_EURO from predict_rentals
where rank_category_per_customer=1;






In [2]:
query='''select * from predict_rentals_step_two'''


predict_rentals=pd.read_sql_query(query, engine)

In [3]:
predict_rentals.set_index('customer_id')




,city,category_name,number_of_rentals,Total_EUR
customer_id,,,,
1,Sasebo,Classics,6,14.94
2,San Bernardino,Sports,5,27.95
3,Athenai,Action,4,20.96
4,Myingyan,Horror,3,12.97
5,Nantou,Classics,7,20.93
...,...,...,...,...
595,Jinzhou,Documentary,4,7.96
596,Patras,Sports,5,16.95
597,Sullana,Foreign,4,16.96


Take the rentals_table and find out which customer did not rent a film in June 2005

Digit 1 means customer rented -
Digit 0 means customer did not rent

599 existing customers and 590 of them rented a film in june


In [4]:
query='''with cte as
(select distinct customer_id,1 as rented_june from customer cust
join rental
using(customer_id)
where month(rental_date) = 6)
select customer_id,rented_june from customer
left join cte
using(customer_id)'''


rentals_june=pd.read_sql_query(query, engine)

In [5]:
#when a customer rented a film in june = 1 if not 0


rentals_june.set_index('customer_id')




,rented_june
customer_id,
1,1.0
2,1.0
3,1.0
5,1.0
7,1.0
...,...
582,1.0
584,1.0
590,1.0


In [6]:
predict_rentals.shape,rentals_june.shape

((599, 5), (599, 2))

Merge the two datafiles: predict_rentals and rentals_june by using customer_id

In [7]:
data_merged=predict_rentals.merge(rentals_june,on='customer_id',how='inner')


In [8]:
data_merged.set_index('customer_id')

,city,category_name,number_of_rentals,Total_EUR,rented_june
customer_id,,,,,
1,Sasebo,Classics,6,14.94,1.0
2,San Bernardino,Sports,5,27.95,1.0
3,Athenai,Action,4,20.96,1.0
4,Myingyan,Horror,3,12.97,1.0
5,Nantou,Classics,7,20.93,1.0
...,...,...,...,...,...
595,Jinzhou,Documentary,4,7.96,1.0
596,Patras,Sports,5,16.95,1.0
597,Sullana,Foreign,4,16.96,1.0


In [9]:
data_merged

,customer_id,city,category_name,number_of_rentals,Total_EUR,rented_june
0,1,Sasebo,Classics,6,14.94,1.0
1,2,San Bernardino,Sports,5,27.95,1.0
2,3,Athenai,Action,4,20.96,1.0
3,4,Myingyan,Horror,3,12.97,1.0
4,5,Nantou,Classics,7,20.93,1.0
...,...,...,...,...,...,...
594,595,Jinzhou,Documentary,4,7.96,1.0
595,596,Patras,Sports,5,16.95,1.0
596,597,Sullana,Foreign,4,16.96,1.0
597,598,Lausanne,Drama,3,14.97,1.0


In [10]:
data_merged.describe()

,customer_id,number_of_rentals,Total_EUR,rented_june
count,599.000000,599.000000,599.000000,590.0
mean,300.000000,4.348915,18.106828,1.0
std,173.060683,1.127400,6.416201,0.0
min,1.000000,2.000000,1.980000,1.0
25%,150.500000,4.000000,13.960000,1.0
50%,300.000000,4.000000,16.970000,1.0
75%,449.500000,5.000000,21.950000,1.0
max,599.000000,9.000000,38.950000,1.0


In [11]:
#Replace the NaNs (customer which did not rent in june) with zero.


data_merged=data_merged.fillna(0)

In [12]:
data_merged.isna().sum()

customer_id          0
city                 0
category_name        0
number_of_rentals    0
Total_EUR            0
rented_june          0
dtype: int64

In [13]:
#Clean the data
data_merged.dtypes

customer_id            int64
city                  object
category_name         object
number_of_rentals      int64
Total_EUR            float64
rented_june          float64
dtype: object

In [14]:
#Convert customer_id and rented june into categoricals


data_merged['customer_id'] = data_merged['customer_id'].astype('object')

#data_merged['rented_june'] = data_merged['rented_june'].astype('object')


In [15]:
data_merged

,customer_id,city,category_name,number_of_rentals,Total_EUR,rented_june
0,1,Sasebo,Classics,6,14.94,1.0
1,2,San Bernardino,Sports,5,27.95,1.0
2,3,Athenai,Action,4,20.96,1.0
3,4,Myingyan,Horror,3,12.97,1.0
4,5,Nantou,Classics,7,20.93,1.0
...,...,...,...,...,...,...
594,595,Jinzhou,Documentary,4,7.96,1.0
595,596,Patras,Sports,5,16.95,1.0
596,597,Sullana,Foreign,4,16.96,1.0
597,598,Lausanne,Drama,3,14.97,1.0


In [16]:
data_merged.select_dtypes('object')



,customer_id,city,category_name
0,1,Sasebo,Classics
1,2,San Bernardino,Sports
2,3,Athenai,Action
3,4,Myingyan,Horror
4,5,Nantou,Classics
...,...,...,...
594,595,Jinzhou,Documentary
595,596,Patras,Sports
596,597,Sullana,Foreign
597,598,Lausanne,Drama


In [17]:
#corr_matrix=predict_rentals.corr(method='pearson')  # default
#fig, ax = plt.subplots(figsize=(10, 8))
#ax = sns.heatmap(corr_matrix, annot=True)
#plt.show()

In [18]:
#for col in predict_rentals.select_dtypes(np.number):
 #   sns.displot(predict_rentals[col])
#    plt.show()

In [19]:
#Train the model


y = data_merged['rented_june']
#y = y.astype('int')
X = data_merged.drop('rented_june', axis=1)


In [20]:
X = pd.get_dummies(X)
X

,number_of_rentals,Total_EUR,customer_id_1,customer_id_2,customer_id_3,customer_id_4,customer_id_5,customer_id_6,customer_id_7,customer_id_8,...,category_name_Drama,category_name_Family,category_name_Foreign,category_name_Games,category_name_Horror,category_name_Music,category_name_New,category_name_Sci-Fi,category_name_Sports,category_name_Travel
0,6,14.94,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,27.95,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4,20.96,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,12.97,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,7,20.93,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,4,7.96,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
595,5,16.95,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
596,4,16.96,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
597,3,14.97,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
#Max iter = 10000!!

classification = LogisticRegression(random_state=42,max_iter=10000) # max_iter
classification.fit(X_train, y_train)

# here the model does not converge, to solve it you can scale the data or increase the max_iter parameter
# the first option will save on processing power.

LogisticRegression(max_iter=10000, random_state=42)

In [23]:
classification.score(X_test, y_test)

0.975

In [24]:
X

,number_of_rentals,Total_EUR,customer_id_1,customer_id_2,customer_id_3,customer_id_4,customer_id_5,customer_id_6,customer_id_7,customer_id_8,...,category_name_Drama,category_name_Family,category_name_Foreign,category_name_Games,category_name_Horror,category_name_Music,category_name_New,category_name_Sci-Fi,category_name_Sports,category_name_Travel
0,6,14.94,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,27.95,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4,20.96,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,12.97,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,7,20.93,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,4,7.96,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
595,5,16.95,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
596,4,16.96,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
597,3,14.97,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [25]:
predictions = classification.predict(X_test)

pd.Series(predictions).value_counts()

1.0    120
dtype: int64

In [26]:
y_test.value_counts()

1.0    117
0.0      3
Name: rented_june, dtype: int64

In [27]:
confusion_matrix(y_test, predictions)

array([[  0,   3],
       [  0, 117]])

In [28]:
X_test

,number_of_rentals,Total_EUR,customer_id_1,customer_id_2,customer_id_3,customer_id_4,customer_id_5,customer_id_6,customer_id_7,customer_id_8,...,category_name_Drama,category_name_Family,category_name_Foreign,category_name_Games,category_name_Horror,category_name_Music,category_name_New,category_name_Sci-Fi,category_name_Sports,category_name_Travel
110,6,17.94,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
419,3,9.97,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
564,3,10.97,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
77,5,24.95,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
181,5,21.95,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,4,20.96,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
340,3,26.97,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
148,4,28.96,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
494,4,26.96,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
y_test

110    1.0
419    1.0
564    1.0
77     1.0
181    1.0
      ... 
399    1.0
340    1.0
148    1.0
494    1.0
439    0.0
Name: rented_june, Length: 120, dtype: float64

In [30]:
X_test,y_test,predictions

(     number_of_rentals  Total_EUR  customer_id_1  customer_id_2  \
 110                  6      17.94              0              0   
 419                  3       9.97              0              0   
 564                  3      10.97              0              0   
 77                   5      24.95              0              0   
 181                  5      21.95              0              0   
 ..                 ...        ...            ...            ...   
 399                  4      20.96              0              0   
 340                  3      26.97              0              0   
 148                  4      28.96              0              0   
 494                  4      26.96              0              0   
 439                  4      16.96              0              0   
 
      customer_id_3  customer_id_4  customer_id_5  customer_id_6  \
 110              0              0              0              0   
 419              0              0            